### Objective

To import the Amazon dataset into the spark environment : **Done**

Do minimum neccary Data analysis of the data : **Done**

Create features, labels dataset using the pipeline, including the transformer, estimator and finally evaluator : **pipeline is yet to be done, rest is complete**

Write the final model and the prediction to the database : **done**

In [1]:
#Starting with import of pyspark and related modules

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import warnings
warnings.filterwarnings("ignore")

from pyspark.ml import *

In [2]:
mllibPath = "mllib/"
externalData = "externalData/"

In [3]:
import shutil

shutil.unpack_archive(externalData+"amazon-business-research-analyst-dataset.zip",extract_dir=externalData)

In [3]:
from pyspark.sql import SparkSession
    
sparkSQL = SparkSession.builder.appName('amazonRA') \
        .config('spark.jars',"/usr/share/java/postgresql-42.2.26.jar") \
        .getOrCreate()

22/11/30 04:08:57 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
22/11/30 04:08:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/30 04:08:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sparkReader = sparkSQL.read

In [5]:
sparkContext = sparkSQL.sparkContext

In [6]:
%%sh
cd externalData/
ls

aidData.csv
amazon-business-research-analyst-dataset.zip
athletes.csv
bls-industry-unemployment.csv
bls-metro-unemployment.csv
bseScripts.json
cleaned_test.csv
Common_FMCG_Labelled_Quarter_Results.csv
companyManagement.csv
companyMarketData.csv
Electricity generation.csv
encoded_cleaned_test.csv
googleMerchPurchases.csv
Laser_Report_2020.xlsx
nyc-collisions-2019-reduced.csv
protests.csv
purchase_data.csv
Quora_answers.csv
selected-indicators-from-world-bank-20002019.zip
skillshare-top-1000-course.zip
stocks.csv
top-pypi-packages-30-days.json
unemployment.csv
updated.csv
us-congress-members.csv
vizheads.csv


In [9]:
cleaned_import = sparkReader.csv(externalData+"cleaned_test.csv",
                                  inferSchema=True,
                                  header=True,
                                 sep=',')

In [10]:
#generating row number for data
from pyspark.sql.window import Window
row_w = Window().orderBy(lit('ID'))
cleaned_RA = cleaned_import.withColumn("rowNum",row_number().over(row_w))

In [11]:
cleaned_RA.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Delivery_person_ID: string (nullable = true)
 |-- Delivery_person_Age: double (nullable = true)
 |-- Delivery_person_Ratings: double (nullable = true)
 |-- Restaurant_latitude: double (nullable = true)
 |-- Restaurant_longitude: double (nullable = true)
 |-- Delivery_location_latitude: double (nullable = true)
 |-- Delivery_location_longitude: double (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Time_Orderd: string (nullable = true)
 |-- Time_Order_picked: string (nullable = true)
 |-- Weather: string (nullable = true)
 |-- Road_traffic_density: string (nullable = true)
 |-- Vehicle_condition: integer (nullable = true)
 |-- Type_of_order: string (nullable = true)
 |-- Type_of_vehicle: string (nullable = true)
 |-- multiple_deliveries: double (nullable = true)
 |-- Festival: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Name:: string (nullable = true)
 |-- rowNum: integer (nullable = false)



In [12]:
cleanedRA = cleaned_RA.dropna()

In [15]:
deliveryPersonFact = cleanedRA.select("ID","Delivery_person_ID","Delivery_person_Age",
                 "Delivery_person_Ratings","Type_of_Vehicle","rowNum")

In [16]:
deliveryPersonFact.show(2)

22/11/30 04:14:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:14:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:14:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:14:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:14:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+------------------+-------------------+-----------------------+---------------+------+
|    ID|Delivery_person_ID|Delivery_person_Age|Delivery_person_Ratings|Type_of_V

In [21]:
deliveryPersonFact.withColumn("dirName",lit("cafolder")).select("*").show(2)

22/11/30 04:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/30 04:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+------------------+-------------------+-----------------------+---------------+------+--------+
|    ID|Delivery_person_ID|Delivery_person_Age|Delivery_person_Ratings|

In [156]:
ordersFact = cleanedRA.select(col("ID").alias("order_ID"),"Restaurant_latitude","Restaurant_longitude",
                              "Delivery_location_latitude","Delivery_location_longitude",
                              "Order_Date","Time_Orderd","Time_Order_picked","Type_of_order",
                              "multiple_deliveries","Road_traffic_density","Weather")

### Learning Objective

To learn about the various transformers available in pyspark and implement a couple in Amazon analysis

Learn about the multiple ML algorithms, and use atleast Regression, Classification and Clustering on Amazon RA data

In [157]:
deliveryPersonFact.show(2)

+------+------------------+-------------------+-----------------------+---------------+
|    ID|Delivery_person_ID|Delivery_person_Age|Delivery_person_Ratings|Type_of_Vehicle|
+------+------------------+-------------------+-----------------------+---------------+
|0x3474|    BANGRES15DEL01|               28.0|                    4.6|     motorcycle|
|0x9420|     JAPRES09DEL03|               23.0|                    4.5|     motorcycle|
+------+------------------+-------------------+-----------------------+---------------+
only showing top 2 rows



In [158]:
#lets first use the imputer to update the Null values.
from pyspark.ml.feature import Imputer

deliveryImputer = Imputer(inputCols=["Delivery_person_Age",
                                     "Delivery_person_Ratings"],
                         outputCols=["DeliveryPersonAge",
                                    "DeliveryPersonRatings"],
                         strategy="mean")

deliveryModel = deliveryImputer.fit(deliveryPersonFact)

In [159]:
deliveryPersonNullRemoved = deliveryModel.transform(deliveryPersonFact). \
                drop("Delivery_person_Age","Delivery_person_Ratings")

In [160]:
deliveryPersonNullRemoved = deliveryPersonNullRemoved.select("ID","Delivery_person_ID","Type_of_Vehicle",
                                round("DeliveryPersonAge",1).alias("DeliveryPersonAge"),
                                round("DeliveryPersonRatings",1).alias("DeliveryPersonRatings"))

In [161]:
deliveryPersonNullRemoved.show(5)

+------+------------------+---------------+-----------------+---------------------+
|    ID|Delivery_person_ID|Type_of_Vehicle|DeliveryPersonAge|DeliveryPersonRatings|
+------+------------------+---------------+-----------------+---------------------+
|0x3474|    BANGRES15DEL01|     motorcycle|             28.0|                  4.6|
|0x9420|     JAPRES09DEL03|     motorcycle|             23.0|                  4.5|
|0x72ee|     JAPRES07DEL03|        scooter|             21.0|                  4.8|
|0xa759|    CHENRES19DEL01|        scooter|             31.0|                  4.6|
|0xc4af|     GOARES04DEL01|     motorcycle|             26.0|                  4.7|
+------+------------------+---------------+-----------------+---------------------+
only showing top 5 rows



In [162]:
#Next we will work on encoding the type of vehicles
deliveryPersonNullRemoved.select("Type_of_vehicle").distinct().show()

+----------------+
| Type_of_vehicle|
+----------------+
|      motorcycle|
|         scooter|
|electric_scooter|
|         bicycle|
+----------------+



In [163]:
#There are three modules to use, OneHotEncoder, VectorIndexer, StringIndexer
from pyspark.ml.feature import OneHotEncoder, VectorIndexer, StringIndexer
stringIndexModel = StringIndexer(inputCols=["Type_of_Vehicle","Delivery_person_ID"],
                                 outputCols=["TypeOfVehicle","DeliveryPersonID"])
delivery_stringIndexed = stringIndexModel.fit(deliveryPersonNullRemoved).transform(deliveryPersonNullRemoved)
delivery_stringIndexed.show(2)

+------+------------------+---------------+-----------------+---------------------+-------------+----------------+
|    ID|Delivery_person_ID|Type_of_Vehicle|DeliveryPersonAge|DeliveryPersonRatings|TypeOfVehicle|DeliveryPersonID|
+------+------------------+---------------+-----------------+---------------------+-------------+----------------+
|0x3474|    BANGRES15DEL01|     motorcycle|             28.0|                  4.6|          0.0|           206.0|
|0x9420|     JAPRES09DEL03|     motorcycle|             23.0|                  4.5|          0.0|           464.0|
+------+------------------+---------------+-----------------+---------------------+-------------+----------------+
only showing top 2 rows



In [164]:
stringIndexOrder = StringIndexer(inputCols=["Weather","Road_traffic_density",
                                            "Type_of_order","multiple_deliveries"]
                                 ,outputCols=["Weather_idx","Road_traffic_density_idx",
                                            "Type_of_order_idx","multiple_deliveries_idx"])
order_stringIndexed = stringIndexOrder.fit(ordersFact).transform(ordersFact)

In [165]:
order_stringIndexed = order_stringIndexed.drop("Weather","Road_traffic_density",
                                            "Type_of_order","multiple_deliveries")

In [166]:
order_stringIndexed.show(2)

+--------+-------------------+--------------------+--------------------------+---------------------------+----------+-----------+-----------------+-----------+------------------------+-----------------+-----------------------+
|order_ID|Restaurant_latitude|Restaurant_longitude|Delivery_location_latitude|Delivery_location_longitude|Order_Date|Time_Orderd|Time_Order_picked|Weather_idx|Road_traffic_density_idx|Type_of_order_idx|multiple_deliveries_idx|
+--------+-------------------+--------------------+--------------------------+---------------------------+----------+-----------+-----------------+-----------+------------------------+-----------------+-----------------------+
|  0x3474|          12.975377|           77.696664|                 13.085377|                  77.806664|29-03-2022|      20:30|            20:35|        1.0|                     1.0|              2.0|                    0.0|
|  0x9420|          26.911378|           75.789034|                 27.001378|              

Casting the time into "timestamp" is simple with cast function. Once that is done, getting the interval between the time stamps is 
done using the regular subtraction. The result is interval timestamp, which can be re-casted in "int", which gives results in seconds.

In [167]:
pickTimeDF = order_stringIndexed.select(col("order_ID").alias('pick_ID'),"Time_Order_picked","Time_orderd",
                           (col("Time_Order_picked").cast("timestamp") - 
                           col("Time_orderd").cast("timestamp")).cast('int').alias('IntervalPickup'))

In [168]:
pickTimeDF.show(5)

+-------+-----------------+-----------+--------------+
|pick_ID|Time_Order_picked|Time_orderd|IntervalPickup|
+-------+-----------------+-----------+--------------+
| 0x3474|            20:35|      20:30|           300|
| 0x9420|            19:45|      19:35|           600|
| 0x72ee|            17:20|      17:15|           300|
| 0xa759|            18:40|      18:25|           900|
| 0xc4af|             9:55|       9:45|           600|
+-------+-----------------+-----------+--------------+
only showing top 5 rows



The order pick up time can be predicted based on the variety of features available. The next steps will be to create the feature/label dataset using the Amazon RA

In [169]:
assemblingData = delivery_stringIndexed.join(order_stringIndexed, delivery_stringIndexed.ID == order_stringIndexed.order_ID)

In [170]:
assemblingData_1 = assemblingData.drop('order_ID')

In [171]:
assemblingData_2 = assemblingData_1.join(pickTimeDF,assemblingData_1.ID == pickTimeDF.pick_ID)

In [172]:
assemblingFinal = assemblingData_2.drop("pick_ID")
assembled = assemblingFinal.drop("Time_Order_picked")
assembled.show(2)

+------+------------------+---------------+-----------------+---------------------+-------------+----------------+-------------------+--------------------+--------------------------+---------------------------+----------+-----------+-----------+------------------------+-----------------+-----------------------+-----------+--------------+
|    ID|Delivery_person_ID|Type_of_Vehicle|DeliveryPersonAge|DeliveryPersonRatings|TypeOfVehicle|DeliveryPersonID|Restaurant_latitude|Restaurant_longitude|Delivery_location_latitude|Delivery_location_longitude|Order_Date|Time_Orderd|Weather_idx|Road_traffic_density_idx|Type_of_order_idx|multiple_deliveries_idx|Time_orderd|IntervalPickup|
+------+------------------+---------------+-----------------+---------------------+-------------+----------------+-------------------+--------------------+--------------------------+---------------------------+----------+-----------+-----------+------------------------+-----------------+-----------------------+--------

In [257]:
assembled.drop("Time_orderd").write. \
    csv(externalData+"amazonRA_encoded.csv",header=True)

In [173]:
featuresDS = assemblingFinal.select("DeliveryPersonID","TypeOfVehicle",
                                   "DeliveryPersonAge","DeliveryPersonRatings",
                                   "Restaurant_latitude","Restaurant_longitude",
                                   "Delivery_location_latitude","Delivery_location_longitude",
                                   "Weather_idx","Type_of_order_idx","multiple_deliveries_idx",
                                   "IntervalPickup")

In [174]:
def writingCSVFiletoDatabase(session, csvFile,dbName,dbTableName):
    
    fileSparkDF = session.read.csv(csvFile,inferSchema=True,header=True)
    try:
        fileSparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')
    

In [175]:
def writingSparkDFtoDatabase(session,sparkDF,dbName,dbTableName):
    
    try:
        sparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')
    

In [176]:
writingSparkDFtoDatabase(sparkSQL,featuresDS,"amazon_ra","featuresDS")

Write Complete


In [177]:
#Lets try feature hasher first

from pyspark.ml.feature import FeatureHasher

featureHashmodel = FeatureHasher(inputCols=["DeliveryPersonID","TypeOfVehicle",
                                   "DeliveryPersonAge","DeliveryPersonRatings",
                                   "Restaurant_latitude","Restaurant_longitude",
                                   "Delivery_location_latitude","Delivery_location_longitude",
                                   "Weather_idx","Type_of_order_idx",
                                    "multiple_deliveries_idx"],
                                outputCol="finalFeatures")

featureHashed = featureHashmodel.transform(featuresDS)

In [178]:
featurecollected =featureHashed.select("IntervalPickup", "finalFeatures")

In [179]:
featurecollected.show(2,truncate=False)

+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|IntervalPickup|finalFeatures                                                                                                                                                |
+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|300           |(262144,[19519,34661,53716,148639,149782,152942,153164,162757,171227,173694,213070],[1.0,206.0,2.0,77.806664,28.0,77.696664,12.975377,13.085377,0.0,0.0,4.6])|
|600           |(262144,[19519,34661,53716,148639,149782,152942,153164,162757,171227,173694,213070],[4.0,464.0,0.0,75.879034,23.0,75.789034,26.911378,27.001378,0.0,0.0,4.5])|
+--------------+-------------------------------------------------------------------------------------------------------------

In [128]:
#lets play a bit scalers
from pyspark.ml.feature import Normalizer, StandardScaler,RobustScaler
from pyspark.ml.feature import MinMaxScaler, MaxAbsScaler, Bucketizer
from pyspark.ml.feature import ElementwiseProduct, VectorAssembler, VectorSizeHint

In [180]:
normed = Normalizer(inputCol='finalFeatures',outputCol="NormedFeatures")

featureNormed = normed.transform(featurecollected)
featureNormed.show(5)

+--------------+--------------------+--------------------+
|IntervalPickup|       finalFeatures|      NormedFeatures|
+--------------+--------------------+--------------------+
|           300|(262144,[19519,34...|(262144,[19519,34...|
|           600|(262144,[19519,34...|(262144,[19519,34...|
|           300|(262144,[19519,34...|(262144,[19519,34...|
|           900|(262144,[19519,34...|(262144,[19519,34...|
|           600|(262144,[19519,34...|(262144,[19519,34...|
+--------------+--------------------+--------------------+
only showing top 5 rows



In [181]:
minMaxed = StandardScaler(withMean=True,inputCol='finalFeatures',outputCol="NormedFeatures")
minFit = minMaxed.fit(featurecollected)
featurMinMaxed = minFit.transform(featurecollected)
featurMinMaxed.show(5)

22/11/29 10:22:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
+--------------+--------------------+--------------------+
|IntervalPickup|       finalFeatures|      NormedFeatures|
+--------------+--------------------+--------------------+
|           300|(262144,[19519,34...|[0.0,0.0,0.0,0.0,...|
|           600|(262144,[19519,34...|[0.0,0.0,0.0,0.0,...|
|           300|(262144,[19519,34...|[0.0,0.0,0.0,0.0,...|
|           900|(262144,[19519,34...|[0.0,0.0,0.0,0.0,...|
|           600|(262144,[19519,34...|[0.0,0.0,0.0,0.0,...|
+--------------+--------------------+--------------------+
only showing top 5 rows



In [185]:
#simple vector assembler
vectAssembled = VectorAssembler(inputCols=["DeliveryPersonID","TypeOfVehicle",
                                   "DeliveryPersonAge","DeliveryPersonRatings",
                                   "Restaurant_latitude","Restaurant_longitude",
                                   "Delivery_location_latitude","Delivery_location_longitude",
                                   "Weather_idx","Type_of_order_idx","multiple_deliveries_idx"],
                               outputCol="features")
assembledVectors = vectAssembled.transform(featuresDS).select("intervalPickup",
                                                             "features")
assembledVectors.show(2,truncate=False)

+--------------+------------------------------------------------------------------------+
|intervalPickup|features                                                                |
+--------------+------------------------------------------------------------------------+
|300           |[206.0,0.0,28.0,4.6,12.975377,77.696664,13.085377,77.806664,1.0,2.0,0.0]|
|600           |[464.0,0.0,23.0,4.5,26.911378,75.789034,27.001378,75.879034,4.0,0.0,0.0]|
+--------------+------------------------------------------------------------------------+
only showing top 2 rows



In [186]:
#Next comes the feature selectors
from pyspark.ml.feature import VarianceThresholdSelector, BucketedRandomProjectionLSH
from pyspark.ml.feature import MinHashLSH, UnivariateFeatureSelector, ChiSqSelector
from pyspark.ml.feature import VectorSlicer, RFormula

In [189]:
selector = ChiSqSelector(numTopFeatures=1, featuresCol="features",
                         outputCol="selectedFeatures", 
                         labelCol="intervalPickup")

result = selector.fit(assembledVectors).transform(assembledVectors)
result.show()

22/11/29 10:37:55 ERROR Executor: Exception in task 0.0 in stage 234.0 (TID 218)
scala.MatchError: [null,[282.0,1.0,23.0,4.8,12.914264,77.6784,12.934264,77.6984,1.0,1.0,1.0]] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.stat.ChiSquareTest$.$anonfun$test$1(ChiSquareTest.scala:82)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:197)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:52)
	at org.apache.spark.scheduler.Task.run(Task.sca

Py4JJavaError: An error occurred while calling o1858.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 234.0 failed 1 times, most recent failure: Lost task 0.0 in stage 234.0 (TID 218) (172.17.0.1 executor driver): scala.MatchError: [null,[282.0,1.0,23.0,4.8,12.914264,77.6784,12.934264,77.6984,1.0,1.0,1.0]] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.stat.ChiSquareTest$.$anonfun$test$1(ChiSquareTest.scala:82)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:197)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.S

In [190]:
selector = UnivariateFeatureSelector(featuresCol="features",
                         outputCol="selectedFeatures", 
                         labelCol="intervalPickup",
                        selectionMode='percentile')
selector.setFeatureType("continuous"). \
            setLabelType("continuous"). \
            setSelectionThreshold(1)
result = selector.fit(assembledVectors).transform(assembledVectors)
result.show()

22/11/29 10:40:21 ERROR Executor: Exception in task 0.0 in stage 243.0 (TID 225)
java.lang.NullPointerException: Null value appeared in non-nullable field:
- field (class: "scala.Double", name: "_1")
- root class: "scala.Tuple2"
If the schema is inferred from a Scala tuple/case class, or a Java bean, please try to use scala.Option[_] or other nullable types (e.g. java.lang.Integer instead of int/scala.Int).
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificSafeProjection.NewInstance_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificSafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.ml.stat.FValueTest$.$anonfun$testRegression$1(FValueTest.scala:128)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2$adapted(RDD.scala:855)
	at o

Py4JJavaError: An error occurred while calling o1894.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 243.0 failed 1 times, most recent failure: Lost task 0.0 in stage 243.0 (TID 225) (172.17.0.1 executor driver): java.lang.NullPointerException: Null value appeared in non-nullable field:
- field (class: "scala.Double", name: "_1")
- root class: "scala.Tuple2"
If the schema is inferred from a Scala tuple/case class, or a Java bean, please try to use scala.Option[_] or other nullable types (e.g. java.lang.Integer instead of int/scala.Int).
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificSafeProjection.NewInstance_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificSafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.ml.stat.FValueTest$.$anonfun$testRegression$1

In [198]:
selector = VarianceThresholdSelector(featuresCol="features",
                         outputCol="selectedFeatures",
                        varianceThreshold=10)

result = selector.fit(assembledVectors).transform(assembledVectors)
result.show(2,truncate=False)

+--------------+------------------------------------------------------------------------+----------------------------------------------------+
|intervalPickup|features                                                                |selectedFeatures                                    |
+--------------+------------------------------------------------------------------------+----------------------------------------------------+
|300           |[206.0,0.0,28.0,4.6,12.975377,77.696664,13.085377,77.806664,1.0,2.0,0.0]|[206.0,28.0,12.975377,77.696664,13.085377,77.806664]|
|600           |[464.0,0.0,23.0,4.5,26.911378,75.789034,27.001378,75.879034,4.0,0.0,0.0]|[464.0,23.0,26.911378,75.789034,27.001378,75.879034]|
+--------------+------------------------------------------------------------------------+----------------------------------------------------+
only showing top 2 rows



In [201]:
data = result.select(col("intervalPickup").alias('label'),
                     col('selectedFeatures').alias('features'))
data.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  300|[206.0,28.0,12.97...|
|  600|[464.0,23.0,26.91...|
+-----+--------------------+
only showing top 2 rows



In [212]:
data_rev1 = result.select(col("intervalPickup").alias('label'),
                     col('features'))
data_rev1.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  300|[206.0,0.0,28.0,4...|
|  600|[464.0,0.0,23.0,4...|
+-----+--------------------+
only showing top 2 rows



In [199]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression, GeneralizedLinearRegression
from pyspark.ml.regression import RandomForestRegressor,GBTRegressor, DecisionTreeRegressionModel
from pyspark.ml.regression import FMRegressor,AFTSurvivalRegression,IsotonicRegression

In [210]:
data.count()

10668

In [209]:
#There are multiple algorithms, so testing in couple of options
(train,test) = data.randomSplit([0.7,0.3])
train.count()

7445

In [217]:
data.select("label").filter(col("label").isNull()).count()

2045

In [226]:
newData = data.dropna('any')

In [227]:
#There are multiple algorithms, so testing in couple of options
(train_r,test_r) = newData.randomSplit([0.7,0.3])
train_r.count()

6081

In [228]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(train_r)

22/11/29 11:10:28 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/11/29 11:10:28 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [229]:

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.0036566411613385284,0.01879865357540184,0.13732133983113926,0.0,0.32296805912230897,0.0]
Intercept: 588.9996549691938
numIterations: 10
objectiveHistory: [0.5000000000000009, 0.4999257507605972, 0.4999025194010279, 0.4998968668985165, 0.4998941239929033, 0.4998938869481651, 0.4998938429985969, 0.4998937858020532, 0.49989373978310614, 0.499893639618846, 0.49989362870555504]
+-------------------+
|          residuals|
+-------------------+
|  -289.910760976033|
|  -289.804992370312|
| -289.8102928715506|
|-289.81998191332434|
|-290.56468033352235|
| -291.1415027040979|
| -291.1915762976206|
|-292.12386732436016|
|-292.68936582265724|
| -292.7871682010974|
|-292.93296690656257|
|-293.28891797913775|
| -293.4363755168928|
|-293.62198629319573|
|-293.49268299042035|
|-293.66875719435154|
| -293.7000566392942|
| -294.4689209899525|
| -294.5247375207067|
| -294.6193102185557|
+-------------------+
only showing top 20 rows

RMSE: 244.614536
r2: 0.000250


In [231]:
lrModel.transform(test_r).show()

+-----+--------------------+-----------------+
|label|            features|       prediction|
+-----+--------------------+-----------------+
|  300|(6,[0,1,4,5],[84....|589.8933801981468|
|  300|(6,[0,1,4,5],[116...|590.0485694524324|
|  300|(6,[0,1,4,5],[433...|591.2855695654977|
|  300|(6,[0,1,4,5],[599...|591.9398583472043|
|  300|(6,[0,1,4,5],[643...|591.9154142731685|
|  300|(6,[0,1,4,5],[731...|592.2598064595048|
|  300|(6,[0,1,4,5],[746...|592.4209886371948|
|  300|(6,[0,1,4,5],[753...|592.4868326830943|
|  300|(6,[0,1,4,5],[789...|592.3015449047399|
|  300|(6,[0,1,4,5],[841...|592.5427857046171|
|  300|(6,[0,1,4,5],[856...|592.7345263983036|
|  300|(6,[0,1,4,5],[106...|593.3690341376779|
|  300|(6,[0,1,4,5],[106...|593.4727164473286|
|  300|(6,[0,1,4,5],[113...|593.6155509337983|
|  300|[0.0,31.0,11.0162...|594.6660171375082|
|  300|[1.0,26.0,18.9942...| 598.251070938581|
|  300|[1.0,35.0,18.9942...|598.4299478625334|
|  300|[2.0,29.0,11.0036...|594.6525278819993|
|  300|[2.0,3

In [247]:
fmregressor = FMRegressor(featuresCol='features',
    labelCol= 'label',
    predictionCol= 'prediction',
    factorSize = 8,
    fitIntercept = True,
    fitLinear = True,
    regParam = 0.2)

In [248]:
fmModel = fmregressor.fit(train_r)

In [251]:
predictionFM = fmModel.transform(test_r).select("label","prediction")

In [252]:
predictionFM.show(2)

+-----+------------------+
|label|        prediction|
+-----+------------------+
|  300|3.8420694336554657|
|  300| 4.824404241603713|
+-----+------------------+
only showing top 2 rows



In [253]:
writingSparkDFtoDatabase(sparkSQL,predictionFM,'amazon_ra','predictionFM')

Write Complete
